In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6,"
os.environ["PYOPENGL_PLATFORM"] = "osmesa"

from pathlib import Path
from omegaconf import OmegaConf
import numpy as np
import cv2
from PIL import Image

import open3d as o3d
import smplx
from smplx.lbs import batch_rodrigues
import trimesh
import torch
import pyrender
import meshplot as mp

from utils.cfg_parser import Config
from utils.utils import makelogger, makepath
from WholeGraspPose.models.fittingop import FittingOP
from WholeGraspPose.trainer import Trainer
from utils.utils import RotConverter
# from WholeGraspPose.models.objectmodel import ObjectModel

import sys
sys.path.insert(0, "/data/3D_dataset/GOAL/")
from tools.objectmodel import ObjectModel
from tools.utils import to_cpu, to_tensor, np2torch, euler
from tools.meshviewer import Mesh

device = 'cuda:0'

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
cfg = Config(
    default_cfg_path='WholeGraspPose/configs/WholeGraspPose.yaml', 
    dataset_dir='/data/3D_dataset/GrabNet/data/GRAB/data/FullGraspPose',
    work_dir="results/pretrained_male_tmp/GraspPose",
    vpe_path="configs/verts_per_edge.npy",
    c_weights_path="WholeGraspPose/configs/rhand_weight.npy",
    exp_name="pretrained_male_tmp",
    gender="male",
    best_net="pretrained_model/male_grasppose_model.pt"
)
save_dir = f"results/pretrained_male_tmp/GraspPose/tmp"
os.makedirs(save_dir, exist_ok=True)

logger = makelogger(makepath(os.path.join(save_dir, f'tmp.log'), isfile=True)).info
grabpose = Trainer(cfg=cfg, inference=True, logger=logger)

2024-09-10 06:52:59,433 - root - INFO - Using 1 CUDA cores [NVIDIA A100 80GB PCIe] for training!
2024-09-10 06:52:59,437 - root - INFO - {'dataset_dir': '/data/3D_dataset/GrabNet/data/GRAB/data/FullGraspPose', 'work_dir': 'results/pretrained_male_tmp/GraspPose', 'vpe_path': 'configs/verts_per_edge.npy', 'c_weights_path': 'WholeGraspPose/configs/rhand_weight.npy', 'exp_name': 'pretrained_male_tmp', 'gender': 'male', 'best_net': 'pretrained_model/male_grasppose_model.pt', 'base_lr': 0.0005, 'batch_size': 128, 'bps_size': 4096, 'cuda_id': 0, 'kl_coef': 0.005, 'latentD': 16, 'log_every_epoch': 10, 'n_epochs': 100, 'n_markers': 512, 'n_neurons': 512, 'n_workers': 8, 'reg_coef': 0.0005, 'seed': 4815, 'try_num': 0, 'use_multigpu': False, 'load_on_ram': False, 'cond_object_height': True, 'motion_intent': False, 'object_class': ['all'], 'robustkl': False, 'kl_annealing': True, 'kl_annealing_epoch': 100, 'marker_weight': 1, 'foot_weight': 0, 'collision_weight': 0, 'consistency_weight': 1, 'dropo

In [3]:
obj_name = '/data/3D_dataset/GrabNet/tools/object_meshes/contact_meshes/camera.ply'
trans = [0.0, 0.0, 0.9] # -1: torch.rand(1) + 0.6
rot = [0.0, 0.0, 90.0] # -1: (np.pi)*torch.rand(1) - np.pi/2 

obj_mesh_base = o3d.io.read_triangle_mesh(obj_name)
obj_mesh_base.compute_vertex_normals()
v_temp = torch.from_numpy(np.expand_dims(np.asarray(obj_mesh_base.vertices), 0)).to(dtype=torch.float32, device=device)
normal_temp = torch.from_numpy(np.expand_dims(np.asarray(obj_mesh_base.vertex_normals), 0)).to(dtype=torch.float32, device=device)

transf_transl = torch.tensor([trans]).to(dtype=torch.float32, device=device)
global_orient = torch.tensor([np.deg2rad(rot)]).to(dtype=torch.float32, device=device)
global_orient_rotmat = batch_rodrigues(global_orient.view(-1, 3)).to(device)   # [N, 3, 3]
global_orient_rotmat_6d = global_orient_rotmat.view(-1, 1, 9)[:, :, :6]

object_verts = torch.matmul(v_temp, global_orient_rotmat)
object_normal = torch.matmul(normal_temp, global_orient_rotmat)

index = np.linspace(0, object_verts.shape[1], num=2048, endpoint=False,retstep=True,dtype=int)[0]
verts_object, normal_object = object_verts[:, index], object_normal[:, index]
feat_object = torch.cat([normal_object, global_orient_rotmat_6d.repeat(1,2048,1)], dim=-1)

with torch.no_grad():
    markers_gen, markers_contact_gen, object_contact_gen = grabpose.full_grasp_net.sample(
        verts_object.permute(0,2,1), 
        feat_object.permute(0,2,1), 
        transf_transl
    )
    markers_gen += transf_transl
    verts_object += transf_transl

fittingop = FittingOP({ 
    'init_lr_h': 0.008, 'num_iter': [300,400,500], 'batch_size': 1,
    'num_markers': 143, 'device': device, 'cfg': cfg,
    'verbose': False, 'hand_ncomps': 24, 'only_rec': False,     # True / False 
    'contact_loss': 'contact', 'logger': logger,
    'data_type': 'markers_143',
})

<ipython-input-3-0bfe87685fb1>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  global_orient = torch.tensor([np.deg2rad(rot)]).to(dtype=torch.float32, device=device)
2024-09-10 06:53:07.415 | INFO     | human_body_prior.tools.model_loader:load_model:97 - Loaded model in eval mode with trained weights: /data/3D_dataset/smpl_related/models/V02_05/snapshots/V02_05_epoch=13_val_loss=0.03.ckpt


In [4]:
obj_mesh = Mesh(filename=obj_name, vscale=1.0)
object_verts = torch.matmul(v_temp, global_orient_rotmat) + transf_transl.unsqueeze(dim=1)
# obj_i = Mesh(vertices=object_verts[0].detach().cpu().numpy(), faces=obj_mesh.faces, vc=[1.0,0.0,0.0])

fittingop.reset()
smplx_results_0 = fittingop.bm_male(
    transl=fittingop.transl_rec, 
    global_orient=RotConverter.rotmat2aa(RotConverter.cont2rotmat(fittingop.glo_rot_rec)),
    body_pose=fittingop.vposer.decode(fittingop.vpose_rec).get('pose_body'),
    return_verts=True
)
# sbj_i = Mesh(vertices=smplx_results.vertices[0].detach().cpu().numpy(), faces=fittingop.bm_male.faces, vc=[0.5,0.5,0.5])

mp_viewer = mp.plot(smplx_results_0.vertices[0].detach().cpu().numpy(), fittingop.bm_male.faces, np.array([0.75,0.75,0.75]))
mp_viewer.add_mesh(object_verts[0].detach().cpu().numpy(), obj_mesh.faces, np.array([1.0,0.0,0.0]))
mp_viewer

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0040337…

/opt/conda/envs/icon/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


In [7]:
fittingop.reset()
markers_fit_gen, smplxparams_gen, loss_gen = fittingop.fitting(
    markers_gen, object_contact_gen, markers_contact_gen, 
    verts_object, normal_object, 'male')

2024-09-10 06:56:00,720 - root - INFO - Save [stage2] iter=445, loss:total: 0.0176 |  rec: 0.0065 |  body_rec: 0.0064 |  marker contact: 0.0002 |  object contact: 0.0004 |  prior contact: 0.0009 |  hand collision: 0.0001 |  object collision: 0.0001 |  foot: 0.0004 |  reg: 0.0002 | , verts_info:hand colli: 4 |  obj colli: 6 |  contact: 3 |  hand markers colli: 0 | 


In [6]:
obj_mesh = Mesh(filename=obj_name, vscale=1.0)
object_verts = torch.matmul(v_temp, global_orient_rotmat) + transf_transl.unsqueeze(dim=1)
# obj_i = Mesh(vertices=object_verts[0].detach().cpu().numpy(), faces=obj_mesh.faces, vc=[1.0,0.0,0.0])

fittingop.reset()
smplx_results = fittingop.bm_male(
    return_verts=True,
    **smplxparams_gen[-1]
)
# sbj_i = Mesh(vertices=smplx_results.vertices[0].detach().cpu().numpy(), faces=fittingop.bm_male.faces, vc=[0.5,0.5,0.5])

mp_viewer = mp.plot(smplx_results.vertices[0].detach().cpu().numpy(), fittingop.bm_male.faces, np.array([0.75,0.75,0.75]))
# mp_viewer.add_mesh(smplx_results_0.vertices[0].detach().cpu().numpy(), fittingop.bm_male.faces, np.array([0.25,0.4,0.8]))
mp_viewer.add_mesh(object_verts[0].detach().cpu().numpy(), obj_mesh.faces, np.array([1.0,0.0,0.0]))
mp_viewer

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0231063…

In [ ]:
obj_mesh = Mesh(filename=obj_name, vscale=1.0)
object_verts = torch.matmul(v_temp, global_orient_rotmat) + transf_transl.unsqueeze(dim=1)
obj_i = Mesh(vertices=object_verts[0].detach().cpu().numpy(), faces=obj_mesh.faces, vc=[1.0,0.0,0.0])

smplx_results = fittingop.bm_male(return_verts=True, **smplxparams_gen[-1])
sbj_i = Mesh(vertices=smplx_results.vertices[0].detach().cpu().numpy(), faces=fittingop.bm_male.faces, vc=[0.5,0.5,0.5])

obj_array = [obj_i, sbj_i]

angle = [-90, 0, 0]
trans = [0.0, -1.0, 0.0]
z_dist = 2.0
a_light = 0.4
d_light = 3.0
w, h = 512, 512

scene = pyrender.Scene(bg_color=[0.0,0.0,0.0,1.0], ambient_light=a_light, name='scene')

pc = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
camera_pose = np.eye(4)
camera_pose[:3, :3] = euler([0.0, 0.0, 0.0], 'xzy')
camera_pose[:3, 3] = np.array([0.0, 0.0, z_dist])
cam = pyrender.Node(name = 'camera', camera=pc, matrix=camera_pose)
scene.add_node(cam)

light = pyrender.light.DirectionalLight(color=np.ones(3), intensity=d_light)
light = pyrender.Node(light=light, matrix=camera_pose)
scene.add_node(light)

for obj in obj_array:
    obj.rot_verts(euler(angle, 'xzy'))
    obj.vertices += trans
    mesh = pyrender.Mesh.from_trimesh(obj)
    scene.add(mesh)

viewer = pyrender.OffscreenRenderer(w, h)
color, depth_buffer = viewer.render(scene)
viewer.delete()

depth = depth_buffer.copy()
mask = depth > 0
color_image = Image.fromarray(np.concatenate([color, (mask[...,np.newaxis]*255.).astype(np.uint8)], axis=-1))
color_image


### Vis Pose

In [ ]:
# load_path = Path("results/pretrained_male_tmp2/GraspPose/bowl/fitting_results.npz")
# data = np.load(load_path, allow_pickle=True)
# index = 0
# # object_mesh = get_object_mesh(object_name, 'GRAB', data['object'][()]['transl'][:n_samples], data['object'][()]['global_orient'][:n_samples], n_samples)

# obj_path = '/data/3D_dataset/GrabNet/data/GRAB/data/tools/object_meshes/contact_meshes/camera.ply'
# transl = torch.tensor(data['object'][()]['transl'][index], dtype=torch.float32, device=device).unsqueeze(0)
# global_orient = torch.tensor(data['object'][()]['global_orient'][index], dtype=torch.float32, device=device).unsqueeze(0)

# obj_mesh = Mesh(filename=obj_path, vscale=1.0)
# obj_m = ObjectModel(v_template=torch.from_numpy(obj_mesh.vertices)).to(device)
# obj_m.faces = obj_mesh.faces
# verts_obj = obj_m(transl=transl, global_orient=global_orient).vertices[0].cpu().numpy()
# obj_i = Mesh(vertices=verts_obj, faces=obj_mesh.faces, vc=[1.0,0.0,0.0])

# # body_mesh, _ = get_body_mesh(data['body'][()], gender, n_samples)

# body_model = smplx.create(
#     '/data/3D_dataset/smpl_related/models', 
#     model_type='smplx', gender='male', ext='npz',
#     num_pca_comps=24, batch_size=1, v_template=None
# ).to(device)

# smplxparams = data['body'][()]
# for key in smplxparams.keys():
#     smplxparams[key] = torch.tensor(smplxparams[key][index], dtype=torch.float32, device=device).unsqueeze(0)
# smplx_results = body_model(return_verts=True, **smplxparams)
# verts = smplx_results.vertices[0].detach().cpu().numpy()
# face = body_model.faces
# sbj_i = Mesh(vertices=verts, faces=face, vc=[0.5,0.5,0.5])

# angle = [-90, 0, 0]
# trans = [0.0, -1.0, 0.0]
# z_dist = 1.0
# a_light = 0.4
# d_light = 3.0
# w, h = 512, 512

# obj_array = [obj_i, sbj_i]

# scene = pyrender.Scene(bg_color=[0.0,0.0,0.0,1.0], ambient_light=a_light, name='scene')

# pc = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
# camera_pose = np.eye(4)
# camera_pose[:3, :3] = euler([0.0, 0.0, 0.0], 'xzy')
# camera_pose[:3, 3] = np.array([0.0, 0.0, z_dist])
# cam = pyrender.Node(name = 'camera', camera=pc, matrix=camera_pose)
# scene.add_node(cam)

# light = pyrender.light.DirectionalLight(color=np.ones(3), intensity=d_light)
# light = pyrender.Node(light=light, matrix=camera_pose)
# scene.add_node(light)

# for obj in obj_array:
#     obj.rot_verts(euler(angle, 'xzy'))
#     obj.vertices += trans
#     mesh = pyrender.Mesh.from_trimesh(obj)
#     scene.add(mesh)

# viewer = pyrender.OffscreenRenderer(w, h)
# color, depth_buffer = viewer.render(scene)
# viewer.delete()

# depth = depth_buffer.copy()
# mask = depth > 0
# color_image = Image.fromarray(np.concatenate([color, (mask[...,np.newaxis]*255.).astype(np.uint8)], axis=-1))
# color_image